In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Processed_27.csv')

In [3]:
df.shape

(2919, 194)

In [4]:
mat = df.corr()

In [7]:
mat.nlargest(20,'SalePrice')['SalePrice']

SalePrice           1.000000
OverallQual         0.790982
GrLivArea           0.708624
GarageCars          0.640409
GarageArea          0.623431
TotalBsmtSF         0.613581
1stFlrSF            0.605852
FullBath            0.560664
BsmtQual_Ex         0.553105
TotRmsAbvGrd        0.533723
YearBuilt           0.522897
YearRemodAdd        0.507101
KitchenQual_Ex      0.504094
Foundation_PConc    0.497734
MasVnrArea          0.472614
GarageYrBlt         0.470177
Fireplaces          0.466929
ExterQual_Gd        0.452466
ExterQual_Ex        0.451164
BsmtFinType1_GLQ    0.434597
Name: SalePrice, dtype: float64

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [11]:
X_train = df.iloc[:1460,:].drop('SalePrice',axis=1)

In [12]:
X_train.shape

(1460, 193)

In [17]:
X_test = df.iloc[1460:,:].drop('SalePrice',axis=1)
y_train = df['SalePrice']

In [18]:
X_test.shape

(1459, 193)

In [19]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [22]:
pca = PCA(.95)     # 95% Variance Retained

In [23]:
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [29]:
X_pca = pca.transform(X_train)
X_test = pca.transform(X_test)

In [30]:
X_pca.shape

(1460, 114)

In [31]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [32]:
xgbr = XGBRegressor()

In [33]:
n_estimators = [125, 150,175,200,225]
max_depth = [2, 3, 4]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3]
base_score=[0.25,0.5,0.75]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [34]:
from sklearn.metrics import mean_squared_log_error

In [42]:
grid_cv = GridSearchCV(estimator=xgbr,
            param_grid=hyperparameter_grid,
            cv=5, 
            scoring = 'neg_mean_absolute_error' ,n_jobs = 4,
            verbose = 10, 
            return_train_score = True,
           )

In [44]:
grid_cv.fit(X_pca,y_train.iloc[:1460])

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   14.4s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   17.0s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   21.9s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   26.4s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   31.7s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   37.2s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   44.3s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   49.8s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  2.3min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:  2.7min
[Parallel(

[18:48:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,...
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'base_score': [0.25, 0.5, 0.75],
                         'booster': ['gbtree', 'gblinear'],
                         'learning_rate': [0.05, 0.1, 0.15, 0.2],
 

In [45]:
grid_cv.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.15, max_delta_step=0,
             max_depth=3, min_child_weight=2, missing=None, n_estimators=150,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [46]:
xgbr_final = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.15, max_delta_step=0,
             max_depth=3, min_child_weight=2, missing=None, n_estimators=150,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [47]:
xgbr_final.fit(X_pca,y_train.iloc[:1460])

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:51:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.15, max_delta_step=0,
             max_depth=3, min_child_weight=2, missing=None, n_estimators=150,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [50]:
y_predict = xgbr_final.predict(X_test)

In [51]:
df_final = pd.DataFrame({'Id':range(1461,2920),'SalePrice':y_predict})

In [52]:
df_final.head()

,Id,SalePrice
0,1461,118339.140625
1,1462,151174.640625
2,1463,184867.125000
3,1464,187580.187500
4,1465,194492.140625


In [53]:
df_final.to_csv('PCA_op.csv',index=False)